<a href="https://colab.research.google.com/github/YusolCho/Basic/blob/main/%5BBAF%5D%5B1%5D%E1%84%85%E1%85%A9%E1%86%AF_%E1%84%89%E1%85%B3%E1%86%BC%E1%84%85%E1%85%B5_%E1%84%8B%E1%85%AD%E1%84%8B%E1%85%B5%E1%86%AB_%E1%84%87%E1%85%AE%E1%86%AB%E1%84%89%E1%85%A5%E1%86%A8_SVM_%E1%84%8E%E1%85%AC%E1%84%8C%E1%85%A9%E1%86%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 패키지, 데이터 로드

In [ ]:
# base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   

# Standardize the data
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# Data Split
import fast_ml
from fast_ml.model_development import train_valid_test_split

# Modeling 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel

# Hyperparameter tuning
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, space_eval

# Feature Selection
import eli5
from eli5.sklearn import PermutationImportance

# Model Evaluation
from sklearn.metrics import f1_score,accuracy_score
from sklearn.metrics import classification_report, confusion_matrix  

# Load Data
data = pd.read_csv('/Users/yusolcho/Desktop/baf/롤 추천 시스템/2차 피드백 이후 파일 모음집/GOLD_DATA.csv')
data.drop('Unnamed: 0' ,axis=1, inplace = True)  # 불필요한 열 삭제 

# 📌 챔피언 군집별로 데이터 나누기

In [ ]:
cluster_1 =['Neeko', 'Ryze', 'Lux', 'Leblanc', 'Lissandra', 'Malzahar', 'Veigar', 'Vex', 
            'Vladimir', 'Viktor', 'Syndra', 'Ahri', 'AurelionSol', 'Azir', 'Annie', 'Anivia', 'Orianna', 
            'Xerath', 'Zoe', 'Ziggs', 'Cassiopeia', 'Kennen', 'Corki', 'Taliyah', 'TwistedFate']
cluster_2 =['Akshan', 'Tristana']
cluster_3 =['Galio', 'Gragas', 'Diana', 'Rumble', 'Malphite', 'Sylas', 'Swain', 'Singed', 
            'Akali', 'Ekko', 'Zac', 'Chogath', 'Kassadin', 'Katarina', 'Fizz']
cluster_4 =['Renekton', 'Sett', 'Yasuo', 'Yone', 'Irelia', 'Zed', 'Jayce', 'KSante', 
            'Qiyana', 'Talon', 'Pantheon']

data_1 = data[data['champName'].isin(cluster_1)]
data_2 = data[data['champName'].isin(cluster_2)]
data_3 = data[data['champName'].isin(cluster_3)]
data_4 = data[data['champName'].isin(cluster_4)]

# 📌 Train, Valid, Test 데이터 Split
* 참고 : https://blog.naver.com/PostView.naver?blogId=siniphia&logNo=221396370872
* https://towardsdatascience.com/how-to-split-data-into-three-sets-train-validation-and-test-and-why-e50d22d3e54c

In [ ]:
# 🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊
X_train, Y_train, X_val, Y_val, X_test, Y_test = train_valid_test_split(data_1, target = 'win', 
                                        train_size=0.8, valid_size=0.1, test_size=0.1)

In [ ]:
print('훈련 데이터 개수 :' ,len(X_train))
print('validation 데이터 개수 :' ,len(X_val))
print('테스트 데이터 개수 :' ,len(X_test))

훈련 데이터 개수 : 4536
validation 데이터 개수 : 567
테스트 데이터 개수 : 568


# 📌 Feature Scaling
* **< 전처리된 데이터의 독립 변수들 >**
* → 'assists', 'kills', 'deaths', 'champExp', 'champName', 'indiPos','lane', 'role', 'gold', 
* → 'tDDealt', 'tDDealtChamp', 'tDTaken','tMKilled', 'turKills', 'division', 'division_1', 
* → 'division_2','division_3', 'division_4', 'lane_BOTTOM', 'lane_JUNGLE', 'lane_MIDDLE','lane_NONE',
* → 'lane_TOP', 'role_CARRY', 'role_DUO', 'role_NONE','role_SOLO', 'role_SUPPORT', 'indiPos_JUNGLE', 
* → 'indiPos_MIDDLE','indiPos_TOP', 'KDA', 'norm_tDDealt', 'norm_tDDealtChamp','norm_tDTaken',
* → 'norm_tMKilled', 'norm_turKills', 'norm_totalAttack','norm_totalKills'


* **< Scaling 거쳐야 하는 변수 >**
* → 'assists', 'kills', 'deaths', 'champExp', 'gold', 'tDDealt', 'tDDealtChamp'
* → 'tDTaken','tMKilled', 'turKills', 'KDA'


* **< Scaling 거치지 않는 변수 >**
* → **numeric 이 아님** : 'champName', 'indiPos','lane', 'role',  , 
* → **one-hot encoding 된 범주형 변수** : 'division', 'division_1', 'division_2','division_3', 'division_4', 'lane_BOTTOM', 'lane_JUNGLE', 'lane_MIDDLE','lane_NONE','lane_TOP', 'role_CARRY', 'role_DUO', 'role_NONE','role_SOLO', 'role_SUPPORT', 'indiPos_JUNGLE', 'indiPos_MIDDLE','indiPos_TOP', 
* → **파생 변수를 생성하기 위한 임시 변수** : 'norm_tDDealt', 'norm_tDDealtChamp','norm_tDTaken', 'norm_tMKilled', 'norm_turKills'
* → **파생 변수를 생성하기 위한 임시 변수** : 'norm_totalAttack', 'norm_totalKills'


* **< Scaling 방법 종류 >**
1. Sklearn의 StandardScaler
2. Sklearn의 MinMaxScaler
3. Sklearn의 RobustScaler

In [ ]:
# 스케일링(스탠다드)
col_names = ['assists', 'kills', 'deaths', 'champExp', 'gold', 'tDDealt', 'tDDealtChamp',
             'tDTaken','tMKilled', 'turKills', 'KDA']  # 스케일링 필요한 변수 정의 
X_train_sub = X_train[col_names]  # 스케일링 필요한 변수만 X_train에서 뽑은 서브 데이터셋 생성
X_val_sub = X_val[col_names]  # 스케일링 필요한 변수만 X_train에서 뽑은 서브 데이터셋 생성
X_test_sub = X_test[col_names]  # 스케일링 필요한 변수만 X_test에서 뽑은 서브 데이터셋 생성
sc = StandardScaler()
X_train_sub_scaled = pd.DataFrame(sc.fit_transform(X_train_sub),
                                  index = X_train_sub.index, columns = X_train_sub.columns) # 서브 데이터셋 스케일링
X_val_sub_scaled = pd.DataFrame(sc.fit_transform(X_val_sub),
                                  index = X_val_sub.index, columns = X_val_sub.columns) # 서브 데이터셋 스케일링
X_test_sub_scaled = pd.DataFrame(sc.fit_transform(X_test_sub),
                                  index = X_test_sub.index, columns = X_test_sub.columns) # 서브 데이터셋 스케일링

# 원래 데이터에서 col_names의 변수를 스케일링 된 데이터셋의 변수로 대체시킨 최종 StandardScaled 데이터 생성(train)
X_train_StandardScaled = X_train.copy()
X_train_StandardScaled['assists'] = X_train_sub_scaled['assists']
X_train_StandardScaled['kills'] = X_train_sub_scaled['kills']
X_train_StandardScaled['deaths'] = X_train_sub_scaled['deaths']
X_train_StandardScaled['champExp'] = X_train_sub_scaled['champExp']
X_train_StandardScaled['gold'] = X_train_sub_scaled['gold']
X_train_StandardScaled['tDDealt'] = X_train_sub_scaled['tDDealt']
X_train_StandardScaled['tDDealtChamp'] = X_train_sub_scaled['tDDealtChamp']
X_train_StandardScaled['tDTaken'] = X_train_sub_scaled['tDTaken']
X_train_StandardScaled['tMKilled'] = X_train_sub_scaled['tMKilled']
X_train_StandardScaled['turKills'] = X_train_sub_scaled['turKills']
X_train_StandardScaled['KDA'] = X_train_sub_scaled['KDA']

# 원래 데이터에서 col_names의 변수를 스케일링 된 데이터셋의 변수로 대체시킨 최종 StandardScaled 데이터 생성 (valid)
X_val_StandardScaled = X_val.copy()
X_val_StandardScaled['assists'] = X_val_sub_scaled['assists']
X_val_StandardScaled['kills'] = X_val_sub_scaled['kills']
X_val_StandardScaled['deaths'] = X_val_sub_scaled['deaths']
X_val_StandardScaled['champExp'] = X_val_sub_scaled['champExp']
X_val_StandardScaled['gold'] = X_val_sub_scaled['gold']
X_val_StandardScaled['tDDealt'] = X_val_sub_scaled['tDDealt']
X_val_StandardScaled['tDDealtChamp'] = X_val_sub_scaled['tDDealtChamp']
X_val_StandardScaled['tDTaken'] = X_val_sub_scaled['tDTaken']
X_val_StandardScaled['tMKilled'] = X_val_sub_scaled['tMKilled']
X_val_StandardScaled['turKills'] = X_val_sub_scaled['turKills']
X_val_StandardScaled['KDA'] = X_val_sub_scaled['KDA']

# 원래 데이터에서 col_names의 변수를 스케일링 된 데이터셋의 변수로 대체시킨 최종 StandardScaled 데이터 생성 (test)
X_test_StandardScaled = X_test.copy()
X_test_StandardScaled['assists'] = X_test_sub_scaled['assists']
X_test_StandardScaled['kills'] = X_test_sub_scaled['kills']
X_test_StandardScaled['deaths'] = X_test_sub_scaled['deaths']
X_test_StandardScaled['champExp'] = X_test_sub_scaled['champExp']
X_test_StandardScaled['gold'] = X_test_sub_scaled['gold']
X_test_StandardScaled['tDDealt'] = X_test_sub_scaled['tDDealt']
X_test_StandardScaled['tDDealtChamp'] = X_test_sub_scaled['tDDealtChamp']
X_test_StandardScaled['tDTaken'] = X_test_sub_scaled['tDTaken']
X_test_StandardScaled['tMKilled'] = X_test_sub_scaled['tMKilled']
X_test_StandardScaled['turKills'] = X_test_sub_scaled['turKills']
X_test_StandardScaled['KDA'] = X_test_sub_scaled['KDA']

In [ ]:
# 모델에 넣을 StandardScaled 된 훈련데이터
X_train_StandardScaled.head(3)

,assists,kills,deaths,champExp,champName,indiPos,lane,role,gold,tDDealt,...,indiPos_MIDDLE,indiPos_TOP,KDA,norm_tDDealt,norm_tDDealtChamp,norm_tDTaken,norm_tMKilled,norm_turKills,norm_totalAttack,norm_totalKills
2793,0.260496,0.248363,-1.119175,0.013373,Ahri,MIDDLE,MIDDLE,SOLO,-0.167603,-0.133602,...,1,0,0.975264,-0.163210,-0.282856,-0.711869,0.325499,0.003366,-0.829071,0.328865
6869,-0.619168,-0.708999,0.233858,-0.060357,Syndra,MIDDLE,MIDDLE,SOLO,-0.114258,0.118415,...,1,0,-0.624254,0.077808,-0.377294,-0.586825,0.825233,-0.819191,-0.880268,0.006042
612,1.140160,0.727044,0.233858,0.243309,Ahri,MIDDLE,MIDDLE,SOLO,0.270441,0.144203,...,1,0,-0.034958,0.102471,0.071483,-0.146055,0.242210,3.293593,3.563702,3.535803


# 📌 모델 훈련 
* **각기 다른 방법으로 스케일링된 3개의 데이터 셋 중 하나를 선택**
* **그 데이터셋에서 훈련에 사용할 변수를 선택**
* **우선 상관계수를 기반으로 select한 훈련데이터: 상관계수가 높은 변수들만 추출함**
## Correlation Coefficient 확인

In [ ]:
#승패와의 상관계수 분석
print('<win 변수와의 상관계수>')
temp = pd.concat([X_train_StandardScaled, Y_train], axis=1)
print(temp.corr()["win"].sort_values(ascending=False))

<win 변수와의 상관계수>
win                  1.000000
KDA                  0.497781
turKills             0.438147
norm_turKills        0.438147
assists              0.380861
kills                0.331684
norm_totalKills      0.316343
gold                 0.277605
champExp             0.250822
norm_totalAttack     0.195827
norm_tDDealtChamp    0.183266
tDDealtChamp         0.183266
norm_tDDealt         0.159571
tDDealt              0.159571
tMKilled             0.084522
norm_tMKilled        0.084522
lane_TOP             0.039585
role_SUPPORT         0.025884
role_DUO             0.025162
division_3           0.023089
division_1           0.018816
lane_BOTTOM          0.015857
indiPos_JUNGLE       0.008265
lane_JUNGLE          0.002041
role_NONE            0.002041
division_2           0.000255
indiPos_MIDDLE      -0.001767
lane_NONE           -0.005215
role_CARRY          -0.009152
indiPos_TOP         -0.014146
lane_MIDDLE         -0.021119
division            -0.032247
role_SOLO           -0.0

In [ ]:
correlation = temp.corr()['win']
variable_list = correlation[(abs(correlation) > 0.15) & (abs(correlation) < 1)].index.tolist()

### 🍊X_train_1, X_val_1, X_test_1은 스탠다드 스케일링 된 X데이터에서 | 상관계수 | >0.15 인 것만 추출한 것 
### 🍊 model_1은 X_train_1에 훈련된, 기본 하이퍼파라미터(rbf, 0.01, 100)인 SVC 모델 

In [ ]:
# 우선 상관계수를 기반으로 select한 훈련데이터: 상관계수 절댓값 0.15이상인 변수들만 추출함
feature_selected = variable_list
X_train_1 = X_train_StandardScaled[feature_selected]
X_val_1 = X_val_StandardScaled[feature_selected]
X_test_1 = X_test_StandardScaled[feature_selected]

In [ ]:
model_1 = sklearn.svm.SVC(kernel='rbf', gamma=0.01, C = 100)  # 모델 객체 생성
model_1.fit(X_train_1, Y_train)  # 데이터에 모델 훈련시키기 

SVC(C=100, gamma=0.01)

* rbf, linear의 정확도가 높게 나왔고 sigmoid는 매우 낮게 나옴 
* poly 는 시간이 오래 걸렸다(중지시켰음)

In [ ]:
# X_val 에 대한 accuracy 구하기
pred = model_1.predict(X_val_1)  # X_test 를 모델에 집어넣고 예측
print('f1 score : ', f1_score(Y_val,pred,average='macro'))
print('accuracy : ', accuracy_score(Y_val,pred))

f1 score :  0.8518513910317025
accuracy :  0.8518518518518519


# 📌 Hyperparameter 튜닝 
**< 튜닝해야하는 파라미터 종류 >**
1. Kernels : 'RBF', 'sigmoid' 'linear' 'poly' 등. 주로 'RBF(가우시안 커널)'이 많이 사용된다. 
2. C : 오차허용정도. 높을수록 오차를 허용하지 않아 정확하지만 과적합의 위험이 있음
3. Gamma : Kernel이 'RBF', 'sigmoid', 'poly'일 때 설정하는 파라미터. 결정경계의 곡률을 조정. 

**< 파라미터 튜닝 방법 >**
1. grid search : 그리드에 지정된 파라미터의 각 조합에 대해 모델을 평가하는 방식
* Sklearn의 GridSearchCV를 사용
2. random search
3. Bayesian optimization

**출처**
* https://www.vebuso.com/2020/03/svm-hyperparameter-tuning-using-gridsearchcv/
* https://medium.com/grabngoinfo/support-vector-machine-svm-hyperparameter-tuning-in-python-a65586289bcb

In [ ]:
# 그리드 변수에 각 파라미터 값 담기(총 4*4*3=48개의 경우의 수, poly는 제외)
param_grid = {'C': [1,33, 66, 100], 
              'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'linear']} 

In [ ]:
# (오래걸림)sklearn의 GridSearchCV를 이용해서 최적의 파라미터 그리드 서치 
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(X_train_1,Y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   1.1s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   1.1s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   1.0s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   1.1s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   1.1s
[CV] END ........................C=1, gamma=1, kernel=linear; total time=   0.6s
[CV] END ........................C=1, gamma=1, kernel=linear; total time=   0.6s
[CV] END ........................C=1, gamma=1, kernel=linear; total time=   0.6s
[CV] END ........................C=1, gamma=1, kernel=linear; total time=   0.7s
[CV] END ........................C=1, gamma=1, kernel=linear; total time=   0.9s
[CV] END .........................C=1, gamma=0.1, kernel=rbf; total time=   0.7s
[CV] END .........................C=1, gamma=0.

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 33, 66, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'linear']},
             verbose=2)

In [ ]:
print(grid.best_estimator_)

SVC(C=33, gamma=0.01)


# 📌 Feature Selection 

## 변수 중요도 알아보기 
* 출처 : https://www.baeldung.com/cs/ml-feature-importance

In [ ]:
# 🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊🍊
# 🍊🍊🍊 model_tuned 에서 위의 GridSearch 결과를 바탕으로 hyperprarameter 값 조정할 것  🍊🍊🍊 🍊🍊🍊 
model_2 = sklearn.svm.SVC(kernel='rbf', gamma=0.01, C=33)  # 모델 객체 생성 
model_2.fit(X_train_1, Y_train)  # 데이터에 모델 훈련시키기 
pred = model_2.predict(X_val_1)  # X_test 를 모델에 집어넣고 예측
print(f1_score(Y_val,pred,average='macro'))
print(accuracy_score(Y_val,pred))

0.8553571428571429
0.855379188712522


### Permutation Feature Importance
* Permutation Importance는 모델을 학습시킨 뒤, 특정 feature의 데이터를 shuffle 했을 때, 검증 데이터 셋에 대한 예측성능을 확인하고 feature importance를 계산한다. 즉, shuffle 했을 때 모델의 성능이 떨어지면, 중요한 feature 인거고 성능이 그대로거나, 좋아지면 중요하지 않은 feature 이다. 예를 들어 축구팀에 있어서 한 선수를 빼고 대회를 진행했을 때 성적이 떨어지면 그 선수가 좋은선수, 성적이 그대로이거나 올라가면 그저 그런 선수거나 팀에 해가되는 선수라는 것이다. 

 

* Permutation Importance 장점

1. 계산이 빠르다
2. 사용범위가 넓고 이해하기 쉽다
3. 일관된 feature의 중요도를 측정할 수 있다.

* 참고 : https://hong-yp-ml-records.tistory.com/51

In [ ]:
perm = PermutationImportance(model_2, scoring = "f1", random_state = 42).fit(X_val_1, Y_val)
eli5.show_weights(perm, top = 80, feature_names = X_val_1.columns.tolist())

Weight,Feature
0.2156 ± 0.0191,champExp
0.2124 ± 0.0306,norm_totalAttack
0.1252 ± 0.0278,turKills
0.0900 ± 0.0289,norm_turKills
0.0497 ± 0.0220,KDA
0.0466 ± 0.0203,assists
0.0277 ± 0.0162,tDDealt
0.0276 ± 0.0124,kills
0.0265 ± 0.0184,norm_totalKills
0.0241 ± 0.0084,norm_tDDealt


* 결과는 Weight와 Feature로 이루어져 있는데 Weight를 보면 +-로 이루어져 있음을 알 수 있다.
* 높이 있을수록 (weight가 +일수록) 중요한 변수이고 밑에 있을수록 (weight가 -일수록) 중요하지 않은 변수이다.

* **상관계수값에 따른 변수 중요도와 차이가 있음!**

In [ ]:
# permutation feature selection을 기반으로 select한 훈련데이터(tDDealt, gold, KDA 삭제)
feature_selected =[
'champExp',
'norm_totalAttack',
'turKills',
'norm_turKills',
'KDA',
'assists',
'tDDealt',
'kills',
'norm_totalKills',
'norm_tDDealt',
'deaths',
'gold',
'tDDealtChamp']
X_train_2 = X_train_1[feature_selected]
X_val_2 = X_val_1[feature_selected]
X_test_2 = X_test_1[feature_selected]

In [ ]:
model_3 = sklearn.svm.SVC(kernel='rbf', gamma=0.01, C=33)  # 모델 객체 생성 
model_3.fit(X_train_2, Y_train)  # 데이터에 모델 훈련시키기 
pred = model_3.predict(X_val_2)  # X_test 를 모델에 집어넣고 예측
print(f1_score(Y_val,pred,average='macro'))
print(accuracy_score(Y_val,pred))

0.8536082345951292
0.8536155202821869


In [ ]:
# X_test 에 대한 accuracy 구하기
pred = model_3.predict(X_val_2)  # X_test 를 모델에 집어넣고 예측
print('f1 score : ', f1_score(Y_val,pred,average='macro'))
print('accuracy : ', accuracy_score(Y_val,pred))

f1 score :  0.8536082345951292
accuracy :  0.8536155202821869


In [ ]:
model_predictions = model_3.predict(X_val_2)
print(confusion_matrix(Y_val,model_predictions))
print(classification_report( Y_val,model_predictions))#Output

[[244  49]
 [ 34 240]]
              precision    recall  f1-score   support

           0       0.88      0.83      0.85       293
           1       0.83      0.88      0.85       274

    accuracy                           0.85       567
   macro avg       0.85      0.85      0.85       567
weighted avg       0.85      0.85      0.85       567

